In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import sklearn as skl
import warnings
import statsmodels.api as sm

from sklearn import datasets, linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.model_selection import cross_val_score, cross_val_predict, ShuffleSplit, validation_curve, cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelBinarizer, RobustScaler, LabelEncoder, scale, MinMaxScaler, PolynomialFeatures
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_curve, roc_auc_score, confusion_matrix
from sklearn.decomposition import PCA
from sklearn import datasets
from sklearn.feature_selection import RFE,SelectFromModel

from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.datasets import make_classification

from xgboost import XGBClassifier 

from scipy.stats import skew, boxcox_normmax
from scipy.special import boxcox1p

warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter('ignore')

In [2]:
#Load Datasets
train_data = pd.read_csv("training_set_values.csv")
train_labels = pd.read_csv("training_set_labels.csv")
test_data = pd.read_csv("test_set_values.csv")

In [3]:
#Join Labels with Train data
train_labels = train_labels.drop(columns='id')
train_data = train_data.join(train_labels)

In [4]:

train_data.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [5]:
test_data.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 41 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   55765 non-null object
gps_height               59400 non-null int64
installer                55745 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59029 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
r

In [7]:
test_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 40 columns):
id                       14850 non-null int64
amount_tsh               14850 non-null float64
date_recorded            14850 non-null object
funder                   13981 non-null object
gps_height               14850 non-null int64
installer                13973 non-null object
longitude                14850 non-null float64
latitude                 14850 non-null float64
wpt_name                 14850 non-null object
num_private              14850 non-null int64
basin                    14850 non-null object
subvillage               14751 non-null object
region                   14850 non-null object
region_code              14850 non-null int64
district_code            14850 non-null int64
lga                      14850 non-null object
ward                     14850 non-null object
population               14850 non-null int64
public_meeting           14029 non-null object
r

In [8]:
#Drop id and recorded from train dataset 
train_data= train_data.drop(columns=['id'])
train_data= train_data.drop(columns=['recorded_by'])

In [9]:
#Drop id and recorded from test dataset 
test_data= test_data.drop(columns=['id'])
test_data= test_data.drop(columns=['recorded_by'])

In [10]:
# Differentiate categorical from numerical variables, even though some are encoded already,
num_train_data = train_data[[
# 'id',
'amount_tsh',
'gps_height',
'num_private',
'population',
]]

other_train_data = train_data[[ 
'longitude',
'latitude',
'date_recorded',
]]
    
categorical_columns = list(set(train_data.columns) - set(num_train_data.columns) - set(other_train_data.columns))
cat_train_data = train_data.loc[:,categorical_columns]

In [11]:
train_data.head()


,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [12]:
train_data_missing = train_data.isna()
train_data_missing = (train_data_missing.sum()/len(train_data))*100
train_data_missing = train_data_missing[train_data_missing!=0]
train_data_missing

funder                6.119529
installer             6.153199
subvillage            0.624579
public_meeting        5.612795
scheme_management     6.526936
scheme_name          47.417508
permit                5.144781
dtype: float64

In [13]:
column_list = cat_train_data.columns.values.tolist()
print(column_list)

['source', 'scheme_management', 'water_quality', 'extraction_type_class', 'quality_group', 'payment_type', 'payment', 'installer', 'waterpoint_type_group', 'waterpoint_type', 'district_code', 'lga', 'wpt_name', 'construction_year', 'management_group', 'source_class', 'extraction_type_group', 'status_group', 'ward', 'subvillage', 'management', 'permit', 'quantity', 'funder', 'region', 'extraction_type', 'scheme_name', 'source_type', 'region_code', 'quantity_group', 'public_meeting', 'basin']


In [14]:
def unique_values(df):
    for column_name in column_list:
        return print(df[column_name].unique())

In [15]:
for column_name in column_list:
      print(column_name,len(train_data[column_name].unique()),train_data[column_name].unique())

source 10 ['spring' 'rainwater harvesting' 'dam' 'machine dbh' 'other'
 'shallow well' 'river' 'hand dtw' 'lake' 'unknown']
scheme_management 13 ['VWC' 'Other' nan 'Private operator' 'WUG' 'Water Board' 'WUA'
 'Water authority' 'Company' 'Parastatal' 'Trust' 'SWC' 'None']
water_quality 8 ['soft' 'salty' 'milky' 'unknown' 'fluoride' 'coloured' 'salty abandoned'
 'fluoride abandoned']
extraction_type_class 7 ['gravity' 'submersible' 'handpump' 'other' 'motorpump' 'wind-powered'
 'rope pump']
quality_group 6 ['good' 'salty' 'milky' 'unknown' 'fluoride' 'colored']
payment_type 7 ['annually' 'never pay' 'per bucket' 'unknown' 'on failure' 'other'
 'monthly']
payment 7 ['pay annually' 'never pay' 'pay per bucket' 'unknown'
 'pay when scheme fails' 'other' 'pay monthly']
installer 2146 ['Roman' 'GRUMETI' 'World vision' ... 'Dina' 'brown' 'SELEPTA']
waterpoint_type_group 6 ['communal standpipe' 'hand pump' 'other' 'improved spring'
 'cattle trough' 'dam']
waterpoint_type 7 ['communal standpipe

In [16]:
#Fields that require intensive review are: installer, funder, scheme_name, ward, lga, wpt_name, subvillage
grouping_col = train_data[[
'installer',
'funder',
'scheme_name',
'ward',
'lga',
'wpt_name',
'subvillage'
]]

In [17]:
for i in grouping_col.columns:
    inspect = grouping_col[i].value_counts()/len(grouping_col)
    print(inspect.head(10))

DWE                   0.292963
Government            0.030724
RWE                   0.020303
Commu                 0.017845
DANIDA                0.017677
KKKT                  0.015118
Hesawa                0.014141
0                     0.013081
TCRS                  0.011902
Central government    0.010471
Name: installer, dtype: float64
Government Of Tanzania    0.152929
Danida                    0.052424
Hesawa                    0.037071
Rwssp                     0.023131
World Bank                0.022710
Kkkt                      0.021667
World Vision              0.020976
Unicef                    0.017795
Tasaf                     0.014764
District Council          0.014192
Name: funder, dtype: float64
K                              0.011481
None                           0.010842
Borehole                       0.009192
Chalinze wate                  0.006818
M                              0.006734
DANIDA                         0.006380
Government                     0.005387

In [18]:
#Data Preparation Beginning 

train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 39 columns):
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   55765 non-null object
gps_height               59400 non-null int64
installer                55745 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59029 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
scheme_management        55523 non-null object


In [19]:
train_data['status_group'].unique()


array(['functional', 'non functional', 'functional needs repair'],
      dtype=object)

In [20]:
train_temp = train_data.copy()

# installer - shortlist of the 5 higher and category other, note that nulls will be included in this criteria.
def replace(x):
    if x in list(['DWE', 'Government','RWE','Commu','DANIDA']):
        return x
    else:
        return 'other'
train_temp.installer = train_temp.installer.map(replace)

# funder - shortlist of the 5 higher and category other, note that nulls will be included in this criteria.
def replace(x):
    if x in list(['Government Of Tanzania',
'Danida',
'Hesawa',
'Rwssp',
'World Bank',
'Kkkt',
'World Vision',
'Unicef',
'Tasaf',
'District Council']):
        return x
    else:
        return 'other'
train_temp.funder = train_temp.funder.map(replace)

# lga - shortlist of the 5 higher and category other, note that nulls will be included in this criteria.
def replace(x):
    if x in list(['Njombe',
'Arusha Rural',
'Moshi Rural',
'Bariadi',
'Rungwe',
'Kilosa',
'Kasulu',
'Mbozi',
'Meru',
'Bagamoyo']):
        return x
    else:
        return 'other'
train_temp.lga = train_temp.lga.map(replace)


#Eliminate scheme_name from the dataset.
train_temp= train_temp.drop(columns=['scheme_name'])

# Eliminate ward from dataset
train_temp= train_temp.drop(columns=['ward'])

# Eliminate wpt_name from dataset#
train_temp= train_temp.drop(columns=['wpt_name'])

# Eliminate subvillage from dataset#
train_temp= train_temp.drop(columns=['subvillage'])

#Choose Between Region and Region_Code and District Code
#train_temp= train_temp.drop(columns=['region'])
#train_temp= train_temp.drop(columns=['region_code'])
#train_temp= train_temp.drop(columns=['district_code'])

#Construction Year, Bin values per decade --- Note: that (-1, 1960] is the same as Unknown
year_bins = [-1, 1960, 1990, 2015]
train_temp['construction_year_bin'] = pd.cut(train_temp.construction_year,year_bins)
train_temp['construction_year_bin'] = train_temp["construction_year_bin"].astype('category') 
train_temp= train_temp.drop(columns=['construction_year'])

#how to treat, date_recorded?
train_temp.date_recorded = pd.to_datetime(train_temp.date_recorded)
train_temp.date_recorded = pd.to_datetime(train_temp.date_recorded)
train_temp.date_recorded = pd.datetime(2014, 1, 1) - pd.to_datetime(train_temp.date_recorded)
train_temp.columns = ['days_since_recorded' if x=='date_recorded' else x for x in train_temp.columns]
train_temp.days_since_recorded = train_temp.days_since_recorded.astype('timedelta64[D]').astype(int)

recorded_bins = [1, 500, 1000, 2000, 4100]
train_temp['days_since_recorded_bin'] = pd.cut(train_temp.days_since_recorded,recorded_bins)
train_temp['days_since_recorded_bin'] = train_temp["days_since_recorded_bin"].astype('category') 

#Is longitude and Latitude relevant to the model
# train_temp= train_temp.drop(columns=['longitude'])
# train_temp= train_temp.drop(columns=['latitude'])

#Decide between waterpoint_type & waterpoint_type_group
# train_temp= train_temp.drop(columns=['waterpoint_type'])
# train_temp= train_temp.drop(columns=['waterpoint_type_group'])

#Decide between extraction_type & extraction_type_group & extraction_type_class
# train_temp= train_temp.drop(columns=['extraction_type'])
# train_temp= train_temp.drop(columns=['extraction_type_group'])
# train_temp= train_temp.drop(columns=['extraction_type_class'])

In [21]:
#Transform Test Dataset 

test_temp = test_data.copy()

# installer - shortlist of the 5 higher and category other, note that nulls will be included in this criteria.
def replace(x):
    if x in list(['DWE', 'Government','RWE','Commu','DANIDA']):
        return x
    else:
        return 'other'
test_temp.installer = test_temp.installer.map(replace)

# funder - shortlist of the 5 higher and category other, note that nulls will be included in this criteria.
def replace(x):
    if x in list(['Government Of Tanzania',
'Danida',
'Hesawa',
'Rwssp',
'World Bank',
'Kkkt',
'World Vision',
'Unicef',
'Tasaf',
'District Council']):
        return x
    else:
        return 'other'
test_temp.funder = test_temp.funder.map(replace)

# lga - shortlist of the 5 higher and category other, note that nulls will be included in this criteria.
def replace(x):
    if x in list(['Njombe',
'Arusha Rural',
'Moshi Rural',
'Bariadi',
'Rungwe',
'Kilosa',
'Kasulu',
'Mbozi',
'Meru',
'Bagamoyo']):
        return x
    else:
        return 'other'
test_temp.lga = test_temp.lga.map(replace)


#Eliminate scheme_name from the dataset.
test_temp= test_temp.drop(columns=['scheme_name'])

# Eliminate ward from dataset
test_temp= test_temp.drop(columns=['ward'])

# Eliminate wpt_name from dataset#
test_temp= test_temp.drop(columns=['wpt_name'])

# Eliminate subvillage from dataset#
test_temp= test_temp.drop(columns=['subvillage'])

#Choose Between Region and Region_Code and District Code
#test_temp= test_temp.drop(columns=['region'])
#test_temp= test_temp.drop(columns=['region_code'])
#test_temp= test_temp.drop(columns=['district_code'])

#Construction Year, Bin values per decade --- Note: that (-1, 1960] is the same as Unknown
year_bins = [-1, 1960, 1990, 2015]
test_temp['construction_year_bin'] = pd.cut(test_temp.construction_year,year_bins)
test_temp['construction_year_bin'] = test_temp["construction_year_bin"].astype('category') 
test_temp= test_temp.drop(columns=['construction_year'])

#how to treat, date_recorded?
test_temp.date_recorded = pd.to_datetime(test_temp.date_recorded)
test_temp.date_recorded = pd.to_datetime(test_temp.date_recorded)
test_temp.date_recorded = pd.datetime(2014, 1, 1) - pd.to_datetime(test_temp.date_recorded)
test_temp.columns = ['days_since_recorded' if x=='date_recorded' else x for x in test_temp.columns]
test_temp.days_since_recorded = test_temp.days_since_recorded.astype('timedelta64[D]').astype(int)

recorded_bins = [1, 500, 1000, 2000, 4100]
test_temp['days_since_recorded_bin'] = pd.cut(test_temp.days_since_recorded,recorded_bins)
test_temp['days_since_recorded_bin'] = test_temp["days_since_recorded_bin"].astype('category') 

#Is longitude and Latitude relevant to the model
# test_temp= test_temp.drop(columns=['longitude'])
# test_temp= test_temp.drop(columns=['latitude'])

#Decide between waterpoint_type & waterpoint_type_group
# test_temp= test_temp.drop(columns=['waterpoint_type'])
# test_temp= test_temp.drop(columns=['waterpoint_type_group'])

#Decide between extraction_type & extraction_type_group & extraction_type_class
# test_temp= test_temp.drop(columns=['extraction_type'])
# test_temp= test_temp.drop(columns=['extraction_type_group'])
# test_temp= test_temp.drop(columns=['extraction_type_class'])

In [22]:
#Transform Train Data Types

train_temp['district_code'] = train_temp["district_code"].astype('category') 
train_temp['region_code'] = train_temp["region_code"].astype('category') 
#target = train_temp[['status_group']]
#train_temp['status_group'] = train_temp["status_group"].astype('category') 

train_temp['funder'] = train_temp["funder"].astype('category') 
train_temp['installer'] = train_temp["installer"].astype('category') 
train_temp['lga'] = train_temp["lga"].astype('category')

In [23]:
#Transform Test Data Types 

test_temp['district_code'] = test_temp["district_code"].astype('category') 
test_temp['region_code'] = test_temp["region_code"].astype('category') 
#target = test_temp[['status_group']]
#test_temp['status_group'] = test_temp["status_group"].astype('category') 

test_temp['funder'] = test_temp["funder"].astype('category') 
test_temp['installer'] = test_temp["installer"].astype('category') 
test_temp['lga'] = test_temp["lga"].astype('category')

In [24]:
len(test_temp.columns)


35

In [25]:
len(train_temp.columns)


36

In [26]:
# # Differentiate categorical from numerical variables

# target = train_temp[['status_group']]

# num_train_temp = train_temp[[
# 'amount_tsh',
# 'gps_height',
# 'num_private',
# 'population',
# 'days_since_recorded']]

# #Other are those fields under review that require further discussion
# other_train_temp = train_temp[[ 
# 'longitude',
# 'latitude',
# 'funder',
# 'lga',
# 'installer',
# ]]
    
# categorical_columns = list(set(train_temp.columns) - set(num_train_temp.columns) - set(other_train_temp.columns))
# cat_train_temp = train_temp.loc[:,categorical_columns]

In [27]:
def numerical_features(df):
    columns = df.columns
    return df._get_numeric_data().columns

def categorical_features(df):
    numerical_columns = numerical_features(df)
    return(list(set(df.columns) - set(numerical_columns)))

def onehot_encode(df):
    numericals = df.get(numerical_features(df))
    new_df = numericals.copy()
    for categorical_column in categorical_features(df):
        new_df = pd.concat([new_df, 
                            pd.get_dummies(df[categorical_column], 
                                           prefix=categorical_column)], 
                           axis=1)
    return new_df

In [28]:
#Baseline Model

#Segregate trainning labels from data set 
train_temp.status_group.replace(['functional', 'non functional','functional needs repair'], [1, 2, 3], inplace=True)
train_labels = train_temp['status_group']
train_temp= train_temp.drop(columns=['status_group'])

In [29]:
#Data frames before the baseline
train_set = onehot_encode(train_temp)
test_set = onehot_encode(test_temp)

In [30]:
#Train Test Before Split 

train_labels
train_set

X_train, X_test, y_train, y_test = train_test_split(train_set, train_labels, train_size=0.8)

rf = RFC(n_estimators = 1000, random_state = 42)

In [31]:

# Randon Forest
rf.fit(X_train, y_train)

rf_score = rf.score(X_test,y_test)

print('Accuracy:', rf_score)

Accuracy: 0.8107744107744108


In [32]:
#KNN neighbors = 5
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=5)  
classifier.fit(X_train, y_train)  

y_pred = classifier.predict(X_test)  

from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred)) 

[[5290 1088  122]
 [1585 2854   65]
 [ 498  167  211]]
              precision    recall  f1-score   support

           1       0.72      0.81      0.76      6500
           2       0.69      0.63      0.66      4504
           3       0.53      0.24      0.33       876

   micro avg       0.70      0.70      0.70     11880
   macro avg       0.65      0.56      0.59     11880
weighted avg       0.69      0.70      0.69     11880



In [33]:
from sklearn.model_selection import train_test_split

In [34]:
X_train, X_test, y_train, y_test = train_test_split(train_set, train_labels, train_size=0.8)


In [106]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [115]:
# test input data with string, int, and symbol-included columns 
from xgboost import XGBRegressor
from xgboost import XGBClassifier 


df = pd.DataFrame({'0': np.random.randint(0, 2, size=100),
                   '[test1]': np.random.uniform(0, 1, size=100),
                   'test2': np.random.uniform(0, 1, size=100),
                  3: np.random.uniform(0, 1, size=100)})

target = df.iloc[:, 0]
predictors = df.iloc[:, 1:]


In [116]:
# basic xgb model
xgb0 = XGBRegressor(objective= 'reg:linear')
xgb0.fit(predictors, target)


xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

ValueError: feature_names may not contain [, ] or <

In [38]:
train_temp = pd.get_dummies(train_temp)

In [39]:
train_temp.head()

,amount_tsh,days_since_recorded,gps_height,longitude,latitude,num_private,population,funder_Danida,funder_District Council,funder_Government Of Tanzania,...,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other,"construction_year_bin_(-1, 1960]","construction_year_bin_(1960, 1990]","construction_year_bin_(1990, 2015]","days_since_recorded_bin_(1, 500]","days_since_recorded_bin_(500, 1000]","days_since_recorded_bin_(1000, 2000]","days_since_recorded_bin_(2000, 4100]"
0,6000.0,1024,1390,34.938093,-9.856322,0,109,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,0.0,301,1399,34.698766,-2.147466,0,280,0,0,0,...,0,0,0,0,0,1,1,0,0,0
2,25.0,310,686,37.460664,-3.821329,0,250,0,0,0,...,0,0,0,0,0,1,1,0,0,0
3,0.0,338,263,38.486161,-11.155298,0,58,0,0,0,...,0,0,0,0,1,0,1,0,0,0
4,0.0,903,0,31.130847,-1.825359,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0


In [43]:
X_train, X_test, y_train, y_test = train_test_split(train_set, train_labels, train_size=0.8)


In [45]:
#Logistic Regression  K-fold: 5

from sklearn import model_selection
from sklearn.model_selection import cross_val_score
kfold = model_selection.KFold(n_splits=5, random_state=42)
modelCV = LogisticRegression()
scoring = 'accuracy'
results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
solver='auto'
multi_class='auto'
print("5-fold cross validation average accuracy: %.3f" % (results.mean()))

5-fold cross validation average accuracy: 0.742


In [46]:
#Logistic Regression  K-fold: 10

from sklearn import model_selection
from sklearn.model_selection import cross_val_score
kfold = model_selection.KFold(n_splits=10, random_state=42)
modelCV = LogisticRegression()
scoring = 'accuracy'
results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
solver='auto'
multi_class='auto'
print("5-fold cross validation average accuracy: %.3f" % (results.mean()))

5-fold cross validation average accuracy: 0.744


In [47]:
#Attempt at modeling with Support Vector Machine

from sklearn.svm import SVC
classifier1= SVC(kernel='rbf', random_state=42)
classifier1.fit(X_train, y_train)

y_pred=classifier1.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("Recall Score:", recall_score(y_test, y_pred))
print("Accuracy Train Score:", classifier1.score(X_train, y_train))
print("Accuracy Test Score:", accuracy_score(y_test, y_pred))

[[5687  697   44]
 [2061 2536   30]
 [ 613  145   67]]
              precision    recall  f1-score   support

           1       0.68      0.88      0.77      6428
           2       0.75      0.55      0.63      4627
           3       0.48      0.08      0.14       825

   micro avg       0.70      0.70      0.70     11880
   macro avg       0.64      0.50      0.51     11880
weighted avg       0.69      0.70      0.67     11880



NameError: name 'recall_score' is not defined

In [58]:
#Logistic Regression Model 2-Final
from sklearn.linear_model import LogisticRegression
regressor=LogisticRegression(C=100, random_state=42)
regressor.fit(X_train, y_train)

y_pred=regressor.predict(X_test)
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, recall_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("Recall Score:", recall_score(y_test, y_pred))
print("Accuracy Train Score:", regressor.score(X_train, y_train))
print("Accuracy Test Score:", accuracy_score(y_test, y_pred))


[[5796  598   34]
 [1541 3057   29]
 [ 596  149   80]]
              precision    recall  f1-score   support

           1       0.73      0.90      0.81      6428
           2       0.80      0.66      0.73      4627
           3       0.56      0.10      0.17       825

   micro avg       0.75      0.75      0.75     11880
   macro avg       0.70      0.55      0.57     11880
weighted avg       0.75      0.75      0.73     11880



ValueError: Target is multiclass but average='binary'. Please choose another average setting.

In [73]:
#LDA Attempt 1: Accuracy 74.5%

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()  
X_train = sc.fit_transform(X_train)  
X_test = sc.transform(X_test)  



In [78]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

lda = LDA(n_components=1)  
X_train = lda.fit_transform(X_train, y_train)  
X_test = lda.transform(X_test)  

In [69]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(max_depth=2, random_state=0)

classifier.fit(X_train, y_train)  
y_pred = classifier.predict(X_test)  

In [70]:
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import accuracy_score

cm = confusion_matrix(y_test, y_pred)  
print(cm)  
print('Accuracy' + str(accuracy_score(y_test, y_pred)))  

[[5991  437    0]
 [1761 2866    0]
 [ 722  103    0]]
Accuracy0.7455387205387205


In [76]:
#PCA Attempt 2
from sklearn.decomposition import PCA
pca_test = PCA()
X_pca_train = pca_test.fit_transform(X_train)
X_pca_test = pca_test.transform(X_test)
variance_component_ratios = pca_test.explained_variance_ratio_
np.set_printoptions(precision=4, suppress=True)
variance_component_ratios

array([1.])

In [80]:
predictions_pca = []
for i in range(1,7):
    pca = PCA(n_components=1)
    X_pca_train = pca.fit_transform(X_train)
    X_pca_test = pca.transform(X_test)
    log_pca_model = LogisticRegression(class_weight = "balanced")
    log_pca_model.fit(X_pca_train,y_train)
    predictions_pca.append(log_pca_model.predict(X_pca_test))

In [81]:
for i in range(1,7):
    print('PCA with Components =', i)
    print(classification_report(y_test,predictions_pca[i-1]))
    print('***************************************************')

PCA with Components = 1
              precision    recall  f1-score   support

           1       0.74      0.86      0.79      6428
           2       0.75      0.71      0.73      4627
           3       0.00      0.00      0.00       825

   micro avg       0.74      0.74      0.74     11880
   macro avg       0.49      0.52      0.51     11880
weighted avg       0.69      0.74      0.71     11880

***************************************************
PCA with Components = 2
              precision    recall  f1-score   support

           1       0.74      0.86      0.79      6428
           2       0.75      0.71      0.73      4627
           3       0.00      0.00      0.00       825

   micro avg       0.74      0.74      0.74     11880
   macro avg       0.49      0.52      0.51     11880
weighted avg       0.69      0.74      0.71     11880

***************************************************
PCA with Components = 3
              precision    recall  f1-score   support

       

In [96]:
#LDA Attempt 2

X_train, X_test, y_train, y_test = train_test_split(train_set, train_labels, train_size=0.8)


In [97]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [98]:
lda_test = LinearDiscriminantAnalysis(n_components=4)
X_lda_train = lda_test.fit_transform(X_train, y_train)
X_lda_test = lda_test.transform(X_test)
log_lda_model = LogisticRegression(class_weight = "balanced")
log_lda_model.fit(X_lda_train,y_train)
predictions_lda = log_lda_model.predict(X_lda_test)
print('LDA with Components =', 2)
print(classification_report(y_test,predictions_lda))
print('***************************************************')

LDA with Components = 2
              precision    recall  f1-score   support

           1       0.76      0.79      0.78      6462
           2       0.75      0.67      0.71      4521
           3       0.31      0.35      0.33       897

   micro avg       0.71      0.71      0.71     11880
   macro avg       0.60      0.61      0.60     11880
weighted avg       0.72      0.71      0.72     11880

***************************************************
